In [1]:
import warnings
warnings.filterwarnings('ignore') 

In [2]:
import pandas as pd
import numpy as np
import qgrid
from numpy import random
import scipy as sc
import numpy.matlib as matlib
import time
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from mlxtend.preprocessing import standardize
from mlxtend.feature_extraction import PrincipalComponentAnalysis as PCA

db = np.loadtxt('segmentationT.data', delimiter = ',')

Y = db[:, 0]
Y = Y.astype(int)
X = db[:, 1:]



In [3]:
def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)

In [4]:
def extract_features(tipo, n):
    
    ext = PCA(n_components=n)
    
    return ext

# Naive bayes

In [5]:
from sklearn.naive_bayes import GaussianNB

In [6]:
def model_NB(nC, impresion = False):

    tiempo_i = time.time()

    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,7])
    recall_list = np.zeros([4,7])
    f_list = np.zeros([4,7])
    TN_list = np.zeros([4])
    FP_list = np.zeros([4])
    FN_list = np.zeros([4])
    TP_list = np.zeros([4])   
    errores = np.zeros(4)
    nb = GaussianNB()
    for j in range(4):
               
        Xest = standardize(X)

        pca = PCA(n_components=nC)
        pca.fit(Xest)
        X_pca = pca.transform(Xest)
        
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_pca, Y, test_size=0.25) # Modificar metodología de validación
#         scaler = MinMaxScaler()#Escala entre 0 y 1
#         Xtrain = scaler.fit_transform(Xtrain)
#         Xtest = scaler.transform(Xtest)
        
        nb.fit(Xtrain, Ytrain)
        pred = nb.predict(Xtest)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_


        #code for calculating confusion matrix 
        _confusion_matrix_ = confusion_matrix(Ytest, pred)
        TN_list[j] = _confusion_matrix_[0][0]
        FP_list[j] = _confusion_matrix_[0][1]
        FN_list[j] = _confusion_matrix_[1][0]
        TP_list[j] = _confusion_matrix_[1][1]
                
        errores[j] = classification_error(pred, Ytest)
          
    if impresion == True:
        cm = confusion_matrix(Ytest, pred)
        columnas = ['Clase %s'%(i) for i in list(ascii_uppercase)[0:len(np.unique(pred))]]
        df_cm = pd.DataFrame(cm,index = columnas, columns = columnas)
        
        grafica = sns.heatmap(df_cm, cmap = 'Blues', annot = True)
        
        grafica.set(xlabel = 'Verdaderos', ylabel = 'Predicciones')
#         print(classification_report(Ytest, pred))

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(time.time()-tiempo_i)

In [7]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    '#Componentes' : pd.Series(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19'])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['#Componentes'], inplace=True)

i = 0
for k in df_types.index:
    Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_NB(int(k), impresion = False)
    df_types["Eficiencia"][i] = Acc
    df_types["Int_Eficiencia"][i] = IntAcc
    df_types["Sensibilidad"][i] = Sen
    df_types["Int_Sensibilidad"][i] = IntSen
    df_types["Precision"][i] = Pre
    df_types["Int_Precision"][i] = IntPre
    df_types["F-Score"][i] = f
    df_types["Int_F-Score"][i] = IntF
    df_types["Error_Prueba"][i] = error
    df_types["Int_error"][i] = stdError
    df_types["Tiempo de ejecución"][i] = tiempo
    
    i += 1
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

In [8]:
qgrid_widget.get_changed_df()

,Eficiencia,Int_Eficiencia,Sensibilidad,Int_Sensibilidad,Precision,Int_Precision,F-Score,Int_F-Score,Error_Prueba,Int_error,Tiempo de ejecución
#Componentes,,,,,,,,,,,
1,0.5216262975778547,0.013090610683755652,0.5270020004193678,0.3790790119426084,0.4440538286647698,0.2967932355556169,0.4568240851615006,0.3195174604194472,0.47837370242214533,0.013090610683755674,0.740673303604126
2,0.5302768166089965,0.013704999582832922,0.5294797500295033,0.35080143796755053,0.5410732001652568,0.23317234625993694,0.48573730422205413,0.27177927661979817,0.4697231833910035,0.013704999582832925,0.7066423892974854
3,0.717560553633218,0.020374681164902875,0.719850020254613,0.3153595313955398,0.7383079411244301,0.23687348883188653,0.6992335783724392,0.2750563677594042,0.282439446366782,0.020374681164902902,0.6986362934112549
4,0.7218858131487889,0.02243721859747913,0.7203184386054661,0.2834229439504573,0.7448403791812906,0.22168855110554986,0.7064420275670057,0.25925791041463325,0.27811418685121103,0.022437218597479124,0.6896276473999023
5,0.75,0.018813635961013448,0.7488826531503173,0.24228470328537544,0.7690371455284595,0.19503025450642553,0.7427068606875318,0.21636796482239096,0.25,0.018813635961013448,0.7596917152404785
6,0.7439446366782007,0.013119161667909246,0.7449542364588144,0.254992604758998,0.7714813373202178,0.2054290760975014,0.7346522932237235,0.22668133994760029,0.25605536332179935,0.013119161667909262,0.6926307678222656
7,0.7802768166089966,0.016549417361339958,0.7727450031135763,0.24449515412868675,0.7884027349373132,0.19882926445303234,0.7645771563445882,0.21623141844776048,0.21972318339100347,0.01654941736133996,0.7096457481384277
8,0.8066608996539792,0.01592142069144116,0.8071447774192055,0.21318676689624458,0.8146221033728455,0.15881858802176438,0.8002564089996614,0.18211541834936254,0.19333910034602075,0.015921420691441143,0.7086446285247803
9,0.8205017301038062,0.025497028811746522,0.8183394186278223,0.19478893395880426,0.8245673798940815,0.162123069255604,0.8146410456836513,0.17340417811803754,0.17949826989619377,0.025497028811746526,0.6876258850097656


# KNN

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


In [84]:
def model_knn(nC, impresion = False):

    tiempo_i = time.time()
    
    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,7])
    recall_list = np.zeros([4,7])
    f_list = np.zeros([4,7])
    TN_list = np.zeros([4])
    FP_list = np.zeros([4])
    FN_list = np.zeros([4])
    TP_list = np.zeros([4])   
    errores = np.zeros(4)
    knn = KNeighborsClassifier(n_neighbors = 1)
    for j in range(4):
        
        Xest = standardize(X)

        pca = PCA(n_components=nC)
        pca.fit(Xest)
        X_pca = pca.transform(Xest)

        Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_pca, Y, test_size=0.25) # Modificar metodología de validación
        
        knn.fit(Xtrain, Ytrain)
        pred = knn.predict(Xtest)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_


        #code for calculating confusion matrix 
        _confusion_matrix_ = confusion_matrix(Ytest, pred)
        TN_list[j] = _confusion_matrix_[0][0]
        FP_list[j] = _confusion_matrix_[0][1]
        FN_list[j] = _confusion_matrix_[1][0]
        TP_list[j] = _confusion_matrix_[1][1]
                
        errores[j] = classification_error(pred, Ytest)
           
    if impresion == True:
        print(confusion_matrix(Ytest, pred))
        print(classification_report(Ytest, pred))

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(time.time()-tiempo_i)


In [87]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    '#Componentes' : pd.Series(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19'])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['#Componentes'], inplace=True)

i = 0
for k in df_types.index:
    Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_knn(int(k), impresion = False)
    df_types["Eficiencia"][i] = Acc
    df_types["Int_Eficiencia"][i] = IntAcc
    df_types["Sensibilidad"][i] = Sen
    df_types["Int_Sensibilidad"][i] = IntSen
    df_types["Precision"][i] = Pre
    df_types["Int_Precision"][i] = IntPre
    df_types["F-Score"][i] = f
    df_types["Int_F-Score"][i] = IntF
    df_types["Error_Prueba"][i] = error
    df_types["Int_error"][i] = stdError
    df_types["Tiempo de ejecución"][i] = tiempo
    
    i += 1
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

In [88]:
qgrid_widget.get_changed_df()

,Eficiencia,Int_Eficiencia,Sensibilidad,Int_Sensibilidad,Precision,Int_Precision,F-Score,Int_F-Score,Error_Prueba,Int_error,Tiempo de ejecución
#Componentes,,,,,,,,,,,
1,0.5583910034602076,0.009584134864256213,0.5635621715728067,0.20864294442526435,0.559417470921723,0.20373155354714406,0.5595370695576732,0.20475200350428127,0.4416089965397924,0.00958413486425619,0.6906278133392334
2,0.6466262975778546,0.018888067094202456,0.6492873722208252,0.17060738809234202,0.6493436920109109,0.16916643345543572,0.6478439798890425,0.1677561542369132,0.3533737024221453,0.018888067094202474,0.7386729717254639
3,0.8421280276816608,0.0028362623375008666,0.8429186998186938,0.12437714595888562,0.8418825507007289,0.12152767835704245,0.8417453428190367,0.1213788700262632,0.1578719723183391,0.002836262337500863,0.7146492004394531
4,0.8931660899653979,0.006039896212702836,0.8913956106528736,0.09345728056855902,0.8919808303943844,0.0787199926176714,0.8907398419017628,0.0821133060017799,0.10683391003460208,0.006039896212702831,0.7616932392120361
5,0.9160899653979239,0.01578571590907844,0.9171199218702445,0.0737603754739281,0.9169106678752978,0.06893498173857676,0.9159266826128657,0.06481286328906441,0.08391003460207612,0.015785715909078426,0.7136499881744385
6,0.9251730103806228,0.009505735716192029,0.9233270060348707,0.07306471089125428,0.9239362033826188,0.060277738506211764,0.9229230516047064,0.062418697170959256,0.07482698961937716,0.009505735716192006,0.7106475830078125
7,0.9385813148788927,0.008693664032111465,0.9387441943128486,0.06890274525554643,0.9388122057503132,0.051557739685155195,0.9380092787933119,0.05524489416669393,0.061418685121107264,0.008693664032111496,0.7176527976989746
8,0.9385813148788926,0.004325259515570956,0.9387294237154089,0.06457017511730857,0.9389807186211926,0.05739300149499956,0.938421835084269,0.057977141747615014,0.06141868512110727,0.004325259515570936,0.7396728992462158
9,0.9515570934256055,0.010235432150691412,0.9530711642947599,0.056083082235506586,0.9515516776122447,0.04392239022783071,0.951834287575091,0.046239326209913896,0.048442906574394456,0.010235432150691379,0.7066428661346436


# Redes Neuronales Artificiales

In [42]:
from sklearn.neural_network import MLPClassifier

In [43]:
def model_MLP(nC, impresion = False):

    mlp=MLPClassifier(activation='tanh',max_iter = 1000,hidden_layer_sizes=(15,15))
        
    tiempo_i = time.time()

    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,7])
    recall_list = np.zeros([4,7])
    f_list = np.zeros([4,7])
    TN_list = np.zeros([4])
    FP_list = np.zeros([4])
    FN_list = np.zeros([4])
    TP_list = np.zeros([4])   
    errores = np.zeros(4)
    for j in range(4):
        
        Xest = standardize(X)

        pca = PCA(n_components=nC)
        pca.fit(Xest)
        X_pca = pca.transform(Xest)
        
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_pca, Y, test_size=0.25) # Modificar metodología de validación
#         scaler = MinMaxScaler()#Escala entre 0 y 1
#         Xtrain = scaler.fit_transform(Xtrain)
#         Xtest = scaler.transform(Xtest)
        
        mlp.fit(Xtrain, Ytrain)
        pred = mlp.predict(Xtest)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_


        #code for calculating confusion matrix 
        _confusion_matrix_ = confusion_matrix(Ytest, pred)
        TN_list[j] = _confusion_matrix_[0][0]
        FP_list[j] = _confusion_matrix_[0][1]
        FN_list[j] = _confusion_matrix_[1][0]
        TP_list[j] = _confusion_matrix_[1][1]
                
        
        errores[j] = classification_error(pred, Ytest)
       
    
    if impresion == True:
        print(confusion_matrix(Ytest, pred))
        print(classification_report(Ytest, pred))

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(time.time()-tiempo_i)
    

In [44]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    '#Componentes' : pd.Series(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19'])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['#Componentes'], inplace=True)

i = 0
for k in df_types.index:
    Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_MLP(int(k), impresion = False)
    df_types["Eficiencia"][i] = Acc
    df_types["Int_Eficiencia"][i] = IntAcc
    df_types["Sensibilidad"][i] = Sen
    df_types["Int_Sensibilidad"][i] = IntSen
    df_types["Precision"][i] = Pre
    df_types["Int_Precision"][i] = IntPre
    df_types["F-Score"][i] = f
    df_types["Int_F-Score"][i] = IntF
    df_types["Error_Prueba"][i] = error
    df_types["Int_error"][i] = stdError
    df_types["Tiempo de ejecución"][i] = tiempo
    
    i += 1
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

In [45]:
qgrid_widget.get_changed_df()

,Eficiencia,Int_Eficiencia,Sensibilidad,Int_Sensibilidad,Precision,Int_Precision,F-Score,Int_F-Score,Error_Prueba,Int_error,Tiempo de ejecución
#Componentes,,,,,,,,,,,
1,0.5484429065743945,0.0072375434821287,0.5427943423025876,0.2908604674188119,0.5195839921680319,0.2521480485825984,0.5141852997814721,0.27367782549266034,0.4515570934256055,0.007237543482128666,9.66078233718872
2,0.6730103806228374,0.007441457843462478,0.6744915525854668,0.20025579123283543,0.6792126712450237,0.15600156094920653,0.6687704792919227,0.16925512740810847,0.3269896193771626,0.007441457843462475,20.433594703674316
3,0.8287197231833909,0.012772338287399106,0.8291989994530974,0.15188491293635725,0.8309067014574639,0.13394853428301082,0.8271458749947161,0.13694544672652176,0.17128027681660898,0.012772338287399139,19.94715118408203
4,0.8858131487889274,0.006356807290959812,0.8879389940110577,0.0974483108144213,0.8887438288377002,0.07978550954023463,0.8862556483989359,0.07988048070119216,0.11418685121107267,0.0063568072909598,21.114214420318604
5,0.9057093425605536,0.008342258443765528,0.9034966575847747,0.10210758772749606,0.9051566368488189,0.08678705043604307,0.902886993240809,0.08885597474306561,0.09429065743944637,0.008342258443765535,21.08218288421631
6,0.9178200692041523,0.007785467128027677,0.9160898968093287,0.09412348196038665,0.9170333304514015,0.07089493018942164,0.9150499759142876,0.07596625191031392,0.08217993079584775,0.00778546712802768,21.121220111846924
7,0.9446366782006921,0.009554810568501076,0.9448236906266351,0.0557281084015938,0.9443029376154615,0.047702495420016845,0.9441086585298702,0.047807853665118534,0.05536332179930795,0.009554810568501092,20.705841541290283
8,0.9403114186851211,0.005802944578286654,0.9385940856942623,0.06826836072571969,0.9386467367580141,0.06105761484971816,0.9380906862024779,0.061487885855792304,0.05968858131487889,0.00580294457828665,21.229318141937256
9,0.9571799307958477,0.003088853126532379,0.9566046066340139,0.047444624253851354,0.9570321959957092,0.04376591094146409,0.9565431707717885,0.04303928180042753,0.042820069204152246,0.0030888531265323748,20.189372301101685


# Random forest

In [9]:
from sklearn.ensemble import RandomForestClassifier # Se llama a la librería del método Random Forest

In [23]:
def model_RF(nC, impresion = False):
        
    tiempo_i = time.time()
    
    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,7])
    recall_list = np.zeros([4,7])
    f_list = np.zeros([4,7])
    TN_list = np.zeros([4])
    FP_list = np.zeros([4])
    FN_list = np.zeros([4])
    TP_list = np.zeros([4])   
    errores = np.zeros(4)
    
    RF = RandomForestClassifier(n_estimators = 20, max_features = 10)
    
    for j in range(4):
        
        if nC == 19:
            Xest = X
        else:
            Xest = standardize(X)

        pca = PCA(n_components=nC)
        pca.fit(Xest)
        X_pca = pca.transform(Xest)
        
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_pca, Y, test_size=0.25) # Modificar metodología de validación
        if nC == 19:
            scaler = MinMaxScaler()#Escala entre 0 y 1
            Xtrain = scaler.fit_transform(Xtrain)
            Xtest = scaler.transform(Xtest)
        
        RF.fit(Xtrain, Ytrain)
        pred = RF.predict(Xtest)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_

        #code for calculating confusion matrix 
        _confusion_matrix_ = confusion_matrix(Ytest, pred)
        TN_list[j] = _confusion_matrix_[0][0]
        FP_list[j] = _confusion_matrix_[0][1]
        FN_list[j] = _confusion_matrix_[1][0]
        TP_list[j] = _confusion_matrix_[1][1]
                
        
        errores[j] = classification_error(pred, Ytest)
       
    
    if impresion == True:
        cm = confusion_matrix(Ytest, pred)
        columnas = ['Clase %s'%(i) for i in list(ascii_uppercase)[0:len(np.unique(pred))]]
        df_cm = pd.DataFrame(cm,index = columnas, columns = columnas)
        
        grafica = sns.heatmap(df_cm, cmap = 'Blues', annot = True)
        
        grafica.set(xlabel = 'Verdaderos', ylabel = 'Predicciones')
#         print(classification_report(Ytest, pred))

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(time.time()-tiempo_i)
    

In [30]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    '#Componentes' : pd.Series(['10','11','12','13','14','15','16','17','18','19'])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['#Componentes'], inplace=True)

i = 0
for k in df_types.index:
    Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_RF(int(k), impresion = False)
    df_types["Eficiencia"][i] = Acc
    df_types["Int_Eficiencia"][i] = IntAcc
    df_types["Sensibilidad"][i] = Sen
    df_types["Int_Sensibilidad"][i] = IntSen
    df_types["Precision"][i] = Pre
    df_types["Int_Precision"][i] = IntPre
    df_types["F-Score"][i] = f
    df_types["Int_F-Score"][i] = IntF
    df_types["Error_Prueba"][i] = error
    df_types["Int_error"][i] = stdError
    df_types["Tiempo de ejecución"][i] = tiempo
    
    i += 1
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

In [31]:
qgrid_widget.get_changed_df()

,Eficiencia,Int_Eficiencia,Sensibilidad,Int_Sensibilidad,Precision,Int_Precision,F-Score,Int_F-Score,Error_Prueba,Int_error,Tiempo de ejecución
#Componentes,,,,,,,,,,,
10,0.9346885813148789,0.00717263146703155,0.9329259163147949,0.05915500341770596,0.93227571988928,0.062425298604145,0.9323589771109264,0.05903283616440076,0.06531141868512111,0.0071726314670315745,1.292175531387329
11,0.9342560553633219,0.005471068616208265,0.932984458595205,0.07062395069427715,0.9309209862004492,0.05870555323167927,0.931540800031156,0.062439412760759136,0.06574394463667821,0.005471068616208272,1.2741599082946777
12,0.9390138408304498,0.006852499791416444,0.9382065999835418,0.05747249401419851,0.937796945267432,0.05478161421275521,0.9377358146457473,0.05406840866912716,0.06098615916955018,0.006852499791416464,1.2551417350769043
13,0.9368512110726643,0.010198811524698606,0.9381867293910625,0.06504144297833275,0.9387189739108128,0.06512364657276418,0.937923070705991,0.06210543498938143,0.06314878892733565,0.010198811524698613,1.2371253967285156
14,0.9524221453287197,0.007080754992969264,0.9529126276876093,0.05141269765849477,0.9530159260009211,0.04410742021339306,0.952552414245253,0.04397809850604053,0.047577854671280284,0.007080754992969248,1.2691552639007568
15,0.948961937716263,0.013090610683755666,0.9499633221060674,0.06131486362532025,0.950479920057709,0.05378126030078357,0.9498215953510573,0.05492824879108592,0.05103806228373702,0.013090610683755673,1.2291195392608643
16,0.9480968858131488,0.007639931545266289,0.9468739215809887,0.05688034876518496,0.9473875919815083,0.05721933348898995,0.9468842997558361,0.05515558503600107,0.05190311418685121,0.0076399315452663034,1.2591447830200195
17,0.9485294117647058,0.014851457364523694,0.9479873842322822,0.06438087662630051,0.9477039223975414,0.062458443640871504,0.9473732517008304,0.06031201822339399,0.051470588235294115,0.014851457364523701,1.2651524543762207
18,0.9498269896193772,0.006356807290959812,0.9492467409809141,0.05887350535780949,0.9496876563800851,0.050206388973029016,0.9490722849534006,0.051884511273289594,0.05017301038062284,0.006356807290959806,1.2731575965881348


In [22]:
qgrid_widget.get_changed_df()

,Eficiencia,Int_Eficiencia,Sensibilidad,Int_Sensibilidad,Precision,Int_Precision,F-Score,Int_F-Score,Error_Prueba,Int_error,Tiempo de ejecución
#Componentes,,,,,,,,,,,
10,0.939446366782007,0.004410916534249827,0.9393986719308945,0.05845700327754237,0.9388982740527929,0.05169332145116938,0.9389409199411011,0.05345786594672784,0.06055363321799308,0.004410916534249811,1.3121936321258545
11,0.9420415224913495,0.006974271408562791,0.9397555413149022,0.059733079855677655,0.9407051251950529,0.05664808890041068,0.9399529200542452,0.056327673613119286,0.05795847750865052,0.0069742714085627615,1.3091914653778076
12,0.9359861591695502,0.004738084407484118,0.9369979658865325,0.06447244262060232,0.9367299883072893,0.05196720222912277,0.936187144771403,0.05365915090834732,0.06401384083044982,0.004738084407484138,1.3011841773986816
13,0.9411764705882353,0.01378665869421214,0.9398163898557674,0.06456634719751204,0.9403913898959241,0.06536720862196893,0.9398460977212851,0.0631537698162972,0.058823529411764705,0.013786658694212132,1.2791645526885986
14,0.9459342560553633,0.007578466897895854,0.9468494599037239,0.05402679072077679,0.9472935548850228,0.05806479516539686,0.9467386557797256,0.05363741356148753,0.054065743944636674,0.007578466897895862,1.2611472606658936
15,0.9506920415224913,0.006756271345939998,0.9502073292310698,0.05982830197581258,0.95079280593159,0.05283823171424102,0.9499950594521226,0.05288499369893114,0.04930795847750865,0.006756271345940013,1.312194585800171
16,0.9437716262975778,0.00739100670010167,0.9450733557123446,0.06133277709886118,0.944784770547569,0.06276068794723283,0.9444066472660797,0.05872355109127994,0.056228373702422146,0.007391006700101673,1.325204610824585
17,0.9537197231833909,0.004950485788174565,0.9549398873333405,0.05271184088964873,0.9544233299715644,0.04765282117403439,0.9540813098880483,0.04523478365999612,0.046280276816609,0.004950485788174567,1.2881720066070557
18,0.9519896193771626,0.005099405762349295,0.9519441342788463,0.05185626092344895,0.9525050432110793,0.04863141664881565,0.9520324793025028,0.04865166314100497,0.04801038062283737,0.0050994057623493065,1.1990911960601807


# Maquinas de Soporte Vectorial con kernel lineal y con kernel RBF.

In [89]:
from sklearn.svm import SVC

In [90]:
def model_SVC(nC, impresion = False):
    
    tiempo_i = time.time()

    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,7])
    recall_list = np.zeros([4,7])
    f_list = np.zeros([4,7])
    TN_list = np.zeros([4])
    FP_list = np.zeros([4])
    FN_list = np.zeros([4])
    TP_list = np.zeros([4])   
    errores = np.zeros(4)
    PorcentajeVS = np.zeros(4)
    
    svc = SVC(gamma=0.1, C=float(300),  kernel='rbf', decision_function_shape='ovo' , probability = True)
    
    for j in range(4):
        
        Xest = standardize(X)

        pca = PCA(n_components=nC)
        pca.fit(Xest)
        X_pca = pca.transform(Xest)
        
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_pca, Y, test_size=0.25) # Modificar metodología de validación
#         scaler = MinMaxScaler()#Escala entre 0 y 1
#         Xtrain = scaler.fit_transform(Xtrain)
#         Xtest = scaler.transform(Xtest)
        
        svc.fit(Xtrain, Ytrain)
        pred = svc.predict(Xtest)
        
        #code for calculating accuracy 
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall 
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision 
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_
        
        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_

        #code for calculating confusion matrix 
        _confusion_matrix_ = confusion_matrix(Ytest, pred)
        TN_list[j] = _confusion_matrix_[0][0]
        FP_list[j] = _confusion_matrix_[0][1]
        FN_list[j] = _confusion_matrix_[1][0]
        TP_list[j] = _confusion_matrix_[1][1]
                
        errores[j] = classification_error(pred, Ytest)
        PorcentajeVS[j] = svc.support_vectors_.shape[0] / Xtrain.shape[0]
    
    if impresion == True:
        print(confusion_matrix(Ytest, pred))
        print(classification_report(Ytest, pred))

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(np.mean(PorcentajeVS)), str(time.time()-tiempo_i)
    

In [93]:
pd.options.mode.chained_assignment = None

randn = np.random.randn
df_types = pd.DataFrame({
    '#Componentes' : pd.Series(['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19'])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""
df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["%Vectores de soporte"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['#Componentes'], inplace=True)

i = 0
for k in df_types.index:
    Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, vs, tiempo = model_SVC(int(k), impresion = False)
    df_types["Eficiencia"][i] = Acc
    df_types["Int_Eficiencia"][i] = IntAcc
    df_types["Sensibilidad"][i] = Sen
    df_types["Int_Sensibilidad"][i] = IntSen
    df_types["Precision"][i] = Pre
    df_types["Int_Precision"][i] = IntPre
    df_types["F-Score"][i] = f
    df_types["Int_F-Score"][i] = IntF
    df_types["Error_Prueba"][i] = error
    df_types["Int_error"][i] = stdError
    df_types["%Vectores de soporte"][i] = vs
    df_types["Tiempo de ejecución"][i] = tiempo
    
    i += 1
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)

In [94]:
qgrid_widget.get_changed_df()

,Eficiencia,Int_Eficiencia,Sensibilidad,Int_Sensibilidad,Precision,Int_Precision,F-Score,Int_F-Score,Error_Prueba,Int_error,%Vectores de soporte,Tiempo de ejecución
#Componentes,,,,,,,,,,,,
1,0.5294117647058822,0.011541231889382655,0.5471530520018137,0.33126905457567185,0.5647127198108707,0.2036852198606715,0.49934807452634195,0.2616442870166283,0.47058823529411764,0.011541231889382636,0.7536085450346419,2.3391966819763184
2,0.6314878892733564,0.01127889689481427,0.6436000162962415,0.23707137843547843,0.6678200698130862,0.1756325431606552,0.6299390442441671,0.18266682565024325,0.3685121107266436,0.01127889689481425,0.6685912240184757,1.989807367324829
3,0.8451557093425606,0.014603756934372098,0.8452326453939553,0.15386026350315601,0.8483595728286393,0.11714031114241077,0.8414824239941417,0.12666965227636595,0.15484429065743943,0.014603756934372087,0.38005196304849886,1.5794382095336914
4,0.865916955017301,0.0061777062530647235,0.8680568429278619,0.13652113233535224,0.8762097910563373,0.10108205204686328,0.8653985930065173,0.09984481501940178,0.13408304498269896,0.006177706253064755,0.3145207852193995,1.5884449481964111
5,0.9065743944636677,0.008736595967441257,0.9040582892708401,0.10710531041858021,0.9095161665025089,0.08090837831031164,0.903192959528612,0.08017862160621642,0.09342560553633218,0.008736595967441243,0.30297344110854507,1.6755256652832031
6,0.9199826989619377,0.008508787012502619,0.9186960409096622,0.09249448253829533,0.9207258944642635,0.07266029465159653,0.9183682820437309,0.07653296097919388,0.08001730103806229,0.008508787012502595,0.2678983833718245,1.507371187210083
7,0.944636678200692,0.009554810568501071,0.9434288012077268,0.06284906486729538,0.9435679450224647,0.051946089768790366,0.9430737548664594,0.05445358719699645,0.05536332179930796,0.009554810568501092,0.25606235565819857,1.5102863311767578
8,0.9459342560553633,0.004474083232175021,0.9449660999249269,0.059103279941058184,0.9450281748758578,0.05233520795850714,0.9444049884921955,0.05122139826428788,0.05406574394463668,0.004474083232175,0.23859699769053117,1.3826203346252441
9,0.9515570934256055,0.007027714882903092,0.9526525551231819,0.057648480408994875,0.9522911499039018,0.04460259648584011,0.951937181989107,0.04696368846033624,0.048442906574394456,0.007027714882903079,0.23094688221709003,1.3462259769439697
